In [169]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',None)
import plotly_express as px

In [217]:
# instead of scraping results, use ergast
results = pd.read_csv('./f1db_csv/results.csv')
races = pd.read_csv('./f1db_csv/races.csv')
status = pd.read_csv('./f1db_csv/status.csv')
drivers = pd.read_csv('./f1db_csv/drivers.csv')
constructors = pd.read_csv('./f1db_csv/constructors.csv')

results_info = results.merge(races,on='raceId').merge(drivers,on='driverId').merge(constructors,on='constructorId')
results_sel = results_info[['raceId','driverId','code','forename','surname','constructorId','name_y','year','round','name_x','statusId','points']]
results_status = results_sel.merge(status,on='statusId')
results_status.columns = ['raceId', 'driverId', 'code', 'forename', 'surname', 'constructorId', 'constructor','year', 'round', 'race', 'statusId','points', 'status']
results_status['driver'] = results_status['forename'] + " " + results_status['surname']
results_status

,raceId,driverId,code,forename,surname,constructorId,constructor,year,round,race,statusId,points,status,driver
0,18,1,HAM,Lewis,Hamilton,1,McLaren,2008,1,Australian Grand Prix,1,10.0,Finished,Lewis Hamilton
1,19,1,HAM,Lewis,Hamilton,1,McLaren,2008,2,Malaysian Grand Prix,1,4.0,Finished,Lewis Hamilton
2,21,1,HAM,Lewis,Hamilton,1,McLaren,2008,4,Spanish Grand Prix,1,6.0,Finished,Lewis Hamilton
3,22,1,HAM,Lewis,Hamilton,1,McLaren,2008,5,Turkish Grand Prix,1,8.0,Finished,Lewis Hamilton
4,23,1,HAM,Lewis,Hamilton,1,McLaren,2008,6,Monaco Grand Prix,1,10.0,Finished,Lewis Hamilton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25655,809,654,\N,Walt,Faulkner,113,Kurtis Kraft,1953,2,Indianapolis 500,117,0.0,+24 Laps,Walt Faulkner
25656,809,804,\N,Johnny,Mantz,113,Kurtis Kraft,1953,2,Indianapolis 500,117,0.0,+24 Laps,Johnny Mantz
25657,736,347,\N,Jo,Bonnier,95,Porsche,1962,8,United States Grand Prix,120,0.0,+21 Laps,Jo Bonnier
25658,803,547,\N,Horace,Gould,87,Cooper,1954,5,British Grand Prix,125,0.0,+46 Laps,Horace Gould


In [215]:
results_status.to_csv('./data/results_status.csv',index=False)

# analyse accident rate with weather conditions

In [163]:
# accident status codes
acc_statusid = [3,4,20,130]
# acc_statusid = [3,4,20,73,82,89,100,104,107,130] # mainly accident, collision, spun off, collision damage
# acc_statusid = [73,82,89,100,104,107] # injured, injury, safety concerns, driver unwell, fatal accident, eye injury have few results.
status[status['statusId'].isin(acc_statusid)]

,statusId,status,accident
2,3,Accident,True
3,4,Collision,True
19,20,Spun off,True
72,73,Injured,True
81,82,Injury,True
88,89,Safety concerns,True
99,100,Driver unwell,True
103,104,Fatal accident,True
106,107,Eye injury,True
127,130,Collision damage,True


In [128]:
results_status['accident'] = results_status['statusId'].isin(acc_statusid)
results_status.head()

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver,accident
0,18,1,Lewis,Hamilton,1,McLaren,2008,1,Australian Grand Prix,1,Finished,Lewis Hamilton,False
1,19,1,Lewis,Hamilton,1,McLaren,2008,2,Malaysian Grand Prix,1,Finished,Lewis Hamilton,False
2,21,1,Lewis,Hamilton,1,McLaren,2008,4,Spanish Grand Prix,1,Finished,Lewis Hamilton,False
3,22,1,Lewis,Hamilton,1,McLaren,2008,5,Turkish Grand Prix,1,Finished,Lewis Hamilton,False
4,23,1,Lewis,Hamilton,1,McLaren,2008,6,Monaco Grand Prix,1,Finished,Lewis Hamilton,False


In [150]:
# individual race and weather
accidents_weather_indiv = results_status.merge(weather,on=['year','round'])
accidents_weather_indiv.head()

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver,accident,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindSpeed
0,989,4,Fernando,Alonso,1,McLaren,2018,1,Australian Grand Prix,1,Finished,Fernando Alonso,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
1,989,838,Stoffel,Vandoorne,1,McLaren,2018,1,Australian Grand Prix,1,Finished,Stoffel Vandoorne,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
2,989,1,Lewis,Hamilton,131,Mercedes,2018,1,Australian Grand Prix,1,Finished,Lewis Hamilton,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
3,989,822,Valtteri,Bottas,131,Mercedes,2018,1,Australian Grand Prix,1,Finished,Valtteri Bottas,False,24.077477,30.915315,997.003604,True,36.324324,3.691892
4,989,840,Lance,Stroll,3,Williams,2018,1,Australian Grand Prix,1,Finished,Lance Stroll,False,24.077477,30.915315,997.003604,True,36.324324,3.691892


In [129]:
acc_count = results_status.groupby(['raceId','race','year','round']).agg({'accident':'sum'}).reset_index()
acc_count[acc_count['accident']>7]

,raceId,race,year,round,accident
17,18,Australian Grand Prix,2008,1,8
61,62,United States Grand Prix,2006,10,10
109,110,Brazilian Grand Prix,2003,3,8
123,124,Australian Grand Prix,2002,1,9
170,171,Italian Grand Prix,2000,14,9
198,199,British Grand Prix,1998,9,10
202,203,Belgian Grand Prix,1998,13,12
210,211,Monaco Grand Prix,1997,5,12
228,229,Monaco Grand Prix,1996,6,13
229,230,Spanish Grand Prix,1996,7,10


In [92]:
acc_count.to_csv("./data/accidents.csv",index=False)

In [131]:
# combine accidents and weather
weather = pd.read_csv('./data/weather.csv')
accidents_weather = acc_count.merge(weather,on=['year','round'])

accidents_weather.head()

,raceId,race,year,round,accident,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindSpeed
0,989,Australian Grand Prix,2018,1,0,24.077477,30.915315,997.003604,True,36.324324,3.691892
1,990,Bahrain Grand Prix,2018,2,0,27.982524,47.363107,1009.494175,False,32.198058,0.958252
2,991,Chinese Grand Prix,2018,3,1,19.446429,24.089286,1018.131250,True,37.019643,1.837500
3,992,Azerbaijan Grand Prix,2018,4,6,16.661404,45.651754,1021.913158,False,25.251754,2.222807
4,993,Spanish Grand Prix,2018,5,3,16.050476,52.286667,1001.541905,True,32.339048,1.952381


In [97]:
accidents_weather.to_csv('./data/accidents_weather.csv',index=False)

In [132]:
display(accidents_weather.groupby('Rainfall').count())
display(accidents_weather.groupby('Rainfall').agg({'accident':'sum'}))

# 69 no rain and 28 rain races
# 82 no rain accidents and 47 rain accidents
# average of 82/69=1.19 no rain, 47/28=1.68 rain

,raceId,race,year,round,accident,AirTemp,Humidity,Pressure,TrackTemp,WindSpeed
Rainfall,,,,,,,,,,
False,69,69,69,69,69,69,69,69,69,69
True,28,28,28,28,28,28,28,28,28,28


,accident
Rainfall,
False,82
True,47


In [144]:
px.violin(accidents_weather,x='year',y='accident',color='Rainfall',hover_data=['round'],points='outliers',box=True)

In [157]:
px.violin(accidents_weather,y='accident',color='Rainfall',hover_data=['race','year','round'],points='all')

In [134]:
# count statuses by year
results_status[(results_status['year'] > 1950) & results_status['statusId'].isin(acc_statusid)].groupby(['year','statusId','status']).count()

raceId  driverId  forename  surname  \
year statusId status                                                  
1951 3        Accident               9         9         9        9   
     20       Spun off               1         1         1        1   
1952 3        Accident               5         5         5        5   
     20       Spun off               1         1         1        1   
1953 3        Accident              13        13        13       13   
...                                ...       ...       ...      ...   
2021 130      Collision damage       3         3         3        3   
2022 3        Accident               5         5         5        5   
     4        Collision              6         6         6        6   
     20       Spun off               1         1         1        1   
     130      Collision damage       6         6         6        6   

                                constructorId  constructor  round  race  \
year statusId status                                                      
1951 3        Accident                      9            9      9     9   
     20       Spun off                      1            1      1     1   
1952 3        Accident                      5            5      5     5   
     20       Spun off                      1            1      1     1   
1953 3        Accident                     13           13     13    13   
...                                       ...          ...    ...   ...   
2021 130      Collision damage              3            3      3     3   
2022 3        Accident                      5            5      5     5   
     4        Collision                     6            6      6     6   
     20       Spun off                      1            1      1     1   
     130      Collision damage              6            6      6     6   

                                driver  accident  
year statusId status                              
1951 3        Accident               9         9  
     20       Spun off               1         1  
1952 3        Accident               5         5  
     20       Spun off               1         1  
1953 3        Accident              13        13  
...                                ...       ...  
2021 130      Collision damage       3         3  
2022 3        Accident               5         5  
     4        Collision              6         6  
     20       Spun off               1         1  
     130      Collision damage       6         6  

[190 rows x 10 columns]

In [91]:
# check races with particular statuses
results_status[results_status['status'] == 'Spun off']

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver,accident
11611,51,1,Lewis,Hamilton,1,McLaren,2007,16,Chinese Grand Prix,20,Spun off,Lewis Hamilton,True
11612,2,5,Heikki,Kovalainen,1,McLaren,2009,2,Malaysian Grand Prix,20,Spun off,Heikki Kovalainen,True
11613,135,8,Kimi,Räikkönen,1,McLaren,2002,12,German Grand Prix,20,Spun off,Kimi Räikkönen,True
11614,188,14,David,Coulthard,1,McLaren,1999,14,European Grand Prix,20,Spun off,David Coulthard,True
11615,199,14,David,Coulthard,1,McLaren,1998,9,British Grand Prix,20,Spun off,David Coulthard,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12397,819,663,Ken,Wharton,148,Frazer Nash,1952,3,Belgian Grand Prix,20,Spun off,Ken Wharton,True
12398,835,797,Jackie,Holmes,159,Olson,1950,3,Indianapolis 500,20,Spun off,Jackie Holmes,True
12399,1020,815,Sergio,Pérez,211,Racing Point,2019,11,German Grand Prix,20,Spun off,Sergio Pérez,True
12400,957,836,Pascal,Wehrlein,209,Manor Marussia,2016,10,British Grand Prix,20,Spun off,Pascal Wehrlein,True


In [53]:
# look up race results by raceid
results_status[results_status['raceId'] == 1024]

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver
831,1024,832,Carlos,Sainz,1,McLaren,2019,15,Singapore Grand Prix,1,Finished,Carlos Sainz
857,1024,846,Lando,Norris,1,McLaren,2019,15,Singapore Grand Prix,1,Finished,Lando Norris
1022,1024,1,Lewis,Hamilton,131,Mercedes,2019,15,Singapore Grand Prix,1,Finished,Lewis Hamilton
1286,1024,822,Valtteri,Bottas,131,Mercedes,2019,15,Singapore Grand Prix,1,Finished,Valtteri Bottas
1486,1024,9,Robert,Kubica,3,Williams,2019,15,Singapore Grand Prix,1,Finished,Robert Kubica
2667,1024,807,Nico,Hülkenberg,4,Renault,2019,15,Singapore Grand Prix,1,Finished,Nico Hülkenberg
2690,1024,817,Daniel,Ricciardo,4,Renault,2019,15,Singapore Grand Prix,1,Finished,Daniel Ricciardo
3136,1024,20,Sebastian,Vettel,6,Ferrari,2019,15,Singapore Grand Prix,1,Finished,Sebastian Vettel
3925,1024,844,Charles,Leclerc,6,Ferrari,2019,15,Singapore Grand Prix,1,Finished,Charles Leclerc
4221,1024,826,Daniil,Kvyat,5,Toro Rosso,2019,15,Singapore Grand Prix,1,Finished,Daniil Kvyat


In [43]:
# look up race results by year/round
results_status[(results_status['year'] == 2021) & (results_status['round'] == 1)].sort_values('constructor')

,raceId,driverId,forename,surname,constructorId,constructor,year,round,race,statusId,status,driver
8818,1052,841,Antonio,Giovinazzi,51,Alfa Romeo,2021,1,Bahrain Grand Prix,11,+1 Lap,Antonio Giovinazzi
4283,1052,8,Kimi,Räikkönen,51,Alfa Romeo,2021,1,Bahrain Grand Prix,1,Finished,Kimi Räikkönen
17251,1052,842,Pierre,Gasly,213,AlphaTauri,2021,1,Bahrain Grand Prix,31,Retired,Pierre Gasly
6945,1052,852,Yuki,Tsunoda,213,AlphaTauri,2021,1,Bahrain Grand Prix,1,Finished,Yuki Tsunoda
13657,1052,4,Fernando,Alonso,214,Alpine F1 Team,2021,1,Bahrain Grand Prix,23,Brakes,Fernando Alonso
8427,1052,839,Esteban,Ocon,214,Alpine F1 Team,2021,1,Bahrain Grand Prix,11,+1 Lap,Esteban Ocon
9433,1052,20,Sebastian,Vettel,117,Aston Martin,2021,1,Bahrain Grand Prix,11,+1 Lap,Sebastian Vettel
5500,1052,840,Lance,Stroll,117,Aston Martin,2021,1,Bahrain Grand Prix,1,Finished,Lance Stroll
3940,1052,844,Charles,Leclerc,6,Ferrari,2021,1,Bahrain Grand Prix,1,Finished,Charles Leclerc
3885,1052,832,Carlos,Sainz,6,Ferrari,2021,1,Bahrain Grand Prix,1,Finished,Carlos Sainz


In [48]:
results_status[results_status['status'] == '+1 Lap'].sort_values(['year'])

,raceId,driverId,forename,surname,constructorId,constructor,round,race,statusId,status,driver
year,,,,,,,,,,,
1950,6,6,6,6,6,6,6,6,6,6,6
1951,7,7,7,7,7,7,7,7,7,7,7
1952,8,8,8,8,8,8,8,8,8,8,8
1953,23,23,23,23,23,23,23,23,23,23,23
1954,19,19,19,19,19,19,19,19,19,19,19
1955,12,12,12,12,12,12,12,12,12,12,12
1956,9,9,9,9,9,9,9,9,9,9,9
1957,10,10,10,10,10,10,10,10,10,10,10
1958,16,16,16,16,16,16,16,16,16,16,16


# analyse tyrelife
# maybe sequence of tyre changes? hard -> medium
note: constraints on number of tyres
sole supplier of tyres

In [188]:
tyre_rank = ['HYPERSOFT','ULTRASOFT','SUPERSOFT','SOFT','MEDIUM','HARD','SUPERHARD']
tyre_rank_dict = {x:i+1 for i,x in enumerate(tyre_rank)}
tyre_rank_dict

{'HYPERSOFT': 1,
 'ULTRASOFT': 2,
 'SUPERSOFT': 3,
 'SOFT': 4,
 'MEDIUM': 5,
 'HARD': 6,
 'SUPERHARD': 7}

In [197]:
tyrelife = pd.read_csv('./data/tyrelife.csv')
tyrelife['Compound_num'] = tyrelife['Compound'].map(tyre_rank_dict)
tyrelife.head()


,Team,Driver,Stint,Compound,TyreLife,round,year,Compound_num
0,Ferrari,RAI,1.0,ULTRASOFT,22.0,1,2018,2.0
1,Ferrari,RAI,2.0,SOFT,40.0,1,2018,4.0
2,Ferrari,VET,1.0,ULTRASOFT,29.0,1,2018,2.0
3,Ferrari,VET,2.0,SOFT,33.0,1,2018,4.0
4,Ferrari,VET,2.0,ULTRASOFT,5.0,1,2018,2.0


In [199]:
# median tyre life per constructor
tyrelife.groupby(['Team','Compound']).agg({'TyreLife':'median'}).reset_index()

,Team,Compound,TyreLife
0,Alfa Romeo Racing,HARD,32.0
1,Alfa Romeo Racing,INTERMEDIATE,19.0
2,Alfa Romeo Racing,MEDIUM,20.0
3,Alfa Romeo Racing,SOFT,16.0
4,Alfa Romeo Racing,UNKNOWN,2.0
...,...,...,...
108,Williams,SOFT,15.0
109,Williams,SUPERSOFT,21.0
110,Williams,ULTRASOFT,13.0
111,Williams,UNKNOWN,1.0


In [222]:
# tyre sequence
tyrelist = tyrelife.sort_values(['year','round','Team','Driver','Stint']).groupby(['year','round','Team','Driver']).agg({'Compound':list}).reset_index()
tyrelist.head()
tyrelist_status = (tyrelist.merge(results_status,left_on=['year','round','Team','Driver'],right_on=['year','round','constructor','code'])
.merge(weather[['year','round','Rainfall']],on=['year','round']))

In [224]:
tyrelist_status

,year,round,Team,Driver,Compound,raceId,driverId,code,forename,surname,constructorId,constructor,race,statusId,points,status,driver,Rainfall
0,2018,1,Ferrari,RAI,"[ULTRASOFT, SOFT]",989,8,RAI,Kimi,Räikkönen,6,Ferrari,Australian Grand Prix,1,15.0,Finished,Kimi Räikkönen,True
1,2018,1,Ferrari,VET,"[ULTRASOFT, SOFT, ULTRASOFT]",989,20,VET,Sebastian,Vettel,6,Ferrari,Australian Grand Prix,1,25.0,Finished,Sebastian Vettel,True
2,2018,1,Force India,OCO,"[ULTRASOFT, SOFT]",989,839,OCO,Esteban,Ocon,10,Force India,Australian Grand Prix,1,0.0,Finished,Esteban Ocon,True
3,2018,1,Force India,PER,"[ULTRASOFT, SOFT]",989,815,PER,Sergio,Pérez,10,Force India,Australian Grand Prix,1,0.0,Finished,Sergio Pérez,True
4,2018,1,Haas F1 Team,GRO,[ULTRASOFT],989,154,GRO,Romain,Grosjean,210,Haas F1 Team,Australian Grand Prix,36,0.0,Wheel,Romain Grosjean,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,2021,22,Mercedes,HAM,"[MEDIUM, HARD]",1073,1,HAM,Lewis,Hamilton,131,Mercedes,Abu Dhabi Grand Prix,1,18.0,Finished,Lewis Hamilton,False
1477,2021,22,Williams,LAT,"[MEDIUM, HARD]",1073,849,LAT,Nicholas,Latifi,3,Williams,Abu Dhabi Grand Prix,3,0.0,Accident,Nicholas Latifi,False
1478,2021,22,Williams,LAT,"[MEDIUM, HARD]",1073,849,LAT,Nicholas,Latifi,3,Williams,Abu Dhabi Grand Prix,3,0.0,Accident,Nicholas Latifi,False
1479,2021,22,Williams,RUS,[MEDIUM],1073,847,RUS,George,Russell,3,Williams,Abu Dhabi Grand Prix,6,0.0,Gearbox,George Russell,False


In [223]:
tyrelist_status.to_csv('./data/tyrelist_status.csv',index=False)

In [207]:
tyrelist_count = tyrelist.groupby(['year',tyrelist['Compound'].map(tuple)]).agg({'round':'count'}).reset_index()
tyrelist_count

,year,Compound,round
0,2018,"(HYPERSOFT,)",3
1,2018,"(HYPERSOFT, HYPERSOFT, ULTRASOFT, SUPERSOFT)",1
2,2018,"(HYPERSOFT, SOFT)",11
3,2018,"(HYPERSOFT, SOFT, ULTRASOFT)",1
4,2018,"(HYPERSOFT, SUPERSOFT)",19
...,...,...,...
291,2021,"(SOFT, SOFT, SOFT, MEDIUM)",5
292,2021,"(WET,)",20
293,2021,"(WET, INTERMEDIATE, MEDIUM, MEDIUM, MEDIUM)",1
294,2021,"(WET, INTERMEDIATE, MEDIUM, SOFT, MEDIUM, MEDIUM)",1


In [208]:
tyrelist_count.to_csv('./data/tyrelist_count.csv',index=False)

In [ ]:
# join to wins and weather